In [4]:
import pandas as pd

%autosave 5

Autosaving every 5 seconds


In [12]:
def data_prep():
    WCDT_mRNA = "./data/NEPC/WCDT_mRNA.tab"
    WCDT_clinical = "./data/NEPC/WCDT_clinical.tsv"
    df_mRNA = pd.read_csv(WCDT_mRNA, delimiter="\t")
    df_clinical = pd.read_csv(WCDT_clinical, delimiter="\t")
    print(df_clinical["Classification #3"])

In [13]:
data_prep()

0      Not small cell
1      Not small cell
2      Not small cell
3      Not small cell
4      Not small cell
5      Not small cell
6          Small cell
7             Exclude
8             Exclude
9             Exclude
10     Not small cell
11     Not small cell
12     Not small cell
13     Not small cell
14            Exclude
15            Exclude
16         Small cell
17            Exclude
18            Exclude
19            Exclude
20     Not small cell
21            Exclude
22         Small cell
23     Not small cell
24     Not-small cell
25     Not small cell
26     Not small cell
27            Exclude
28     Not small cell
29            Exclude
            ...      
169    Not small cell
170           Exclude
171        Small Cell
172    Not small cell
173           Exclude
174    Not small cell
175           Exclude
176    Not small cell
177    Not small cell
178           Exclude
179    Not small cell
180    Not small cell
181           Exclude
182    Not small cell
183       